In [1]:
import pandas as pd
import numpy as np

In [2]:
filename='report_2016_09_01-2016_12_01'

In [4]:
!ls -l

total 16121840
-rw-rw-r-- 1 sean sean       88495 Jan  3 01:33 crealytics.ipynb
-rw-rw-r-- 1 sean sean         275 Jan  2 22:17 crealytics.py
-rw-rw-r-- 1 sean sean     2750153 Jan  2 22:17 report_2016_02_01-2016_12_01_10000.csv
-rw-rw-r-- 1 sean sean 12575272973 Jan  2 22:11 report_2016_02_01-2016_12_01.csv
-rw-rw-r-- 1 sean sean        8715 Jan  3 01:47 report_2016_02_01-2016_12_01_datadict.csv
-rw-rw-r-- 1 sean sean  3930615902 Jan  2 22:22 report_2016_09_01-2016_12_01.csv


In [2]:
df_test=pd.read_csv('report_2016_02_01-2016_12_01.csv.gz',nrows=100000)

In [24]:
reader = pd.read_csv('report_2016_02_01-2016_12_01.csv', chunksize=1000000)

In [28]:
cat_types

array(['Ad group ID', 'Ad group', 'Brand', 'Campaign ID', 'Campaign',
       'Category (1st level)', 'Category (2nd level)',
       'Category (3rd level)', 'Category (4th level)',
       'Category (5th level)', 'Country/Territory', 'Custom label 0',
       'Custom label 1', 'Custom label 2', 'Custom label 3',
       'Custom label 4', 'Device', 'Item Id', 'Product type (1st level)',
       'Product type (2nd level)', 'Product type (3rd level)',
       'Product type (4th level)', 'Product type (5th level)'], dtype=object)

In [29]:
dfs=[]
for chunk in reader:
    df=cast_categ(chunk,cat_types)
    dfs.append(cast_categ)

/home/sean/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (13,26) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/sean/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [2]:
from collections import OrderedDict


def do_lookup(cat, typ):
    lookups[cat]={}
    
    def lookup( val):
        if val not in lookups[cat]:
            lookups[cat][val]=typ(len(lookups[cat]))
        return lookups[cat][val]
    return lookup

In [3]:
data_dict=pd.read_excel('datadict.xlsx',usecols=range(5), index_col=1)
id_types=data_dict.index[data_dict.data_class=='id'].values
data_dict.loc[data_dict.data_type.notnull(), 'data_type'] = \
    data_dict.loc[data_dict.data_type.notnull(), 'data_type'].map(eval)

In [4]:
lookups={}
converters={id:do_lookup(id, data_dict.at[id, 'data_type']) for id in id_types  }
converters['CTR'] = lambda(x): np.float32(x[:-1])/100
converters['Conv. rate'] =  lambda(x): np.float32(x[:-1])/100
converters['Total conv. value'] =  lambda(x): np.float32(x.replace(',',''))

parse_dates=['Day']
dtype={ nam: data_dict.at[nam, 'data_type']  for nam in data_dict.index[data_dict.data_type.notnull()]}
# http://stackoverflow.com/a/1176179


In [5]:
df3=pd.read_csv('report_2016_09_01-2016_12_01.csv', 
                converters=converters, parse_dates=parse_dates, dtype=dtype)

KeyboardInterrupt: 

 wc report_2016_09_01-2016_12_01.csv 
  14466685  282408421 3930615902 report_2016_09_01-2016_12_01.csv


In [6]:
reader = pd.read_csv('report_2016_09_01-2016_12_01.csv', converters=converters, 
                     parse_dates=parse_dates, dtype=dtype, chunksize=1000000)

In [7]:
dfs=[]
for i,ch in enumerate(reader):
    print i
    ch.to_csv('report_2016_09_01-2016_12_01_{:02d}.csv'.format(i),index=False)
    dfs.append(ch)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [9]:
df_mg=pd.concat(dfs)

In [11]:
del dfs

In [10]:
df_mg.to_pickle('report_2016_09_01-2016_12_01.pkl')

In [20]:
df_mg.dtypes

Ad group ID                          int64
Ad group                             int64
Brand                                int64
Campaign ID                          int64
Campaign                             int64
Category (1st level)                 int64
Category (2nd level)                 int64
Category (3rd level)                 int64
Category (4th level)                 int64
Category (5th level)                 int64
Clicks                              uint16
Conv. rate                         float64
Conversions                         uint16
Total conv. value                  float64
Cost                               float32
Country/Territory                    int64
Cross-device conv.                  uint16
CTR                                float64
Custom label 0                       int64
Custom label 1                       int64
Custom label 2                       int64
Custom label 3                       int64
Custom label 4                       int64
Day        

In [12]:
lookups_df = pd.concat({id: pd.Series(lookups[id]) for id in lookups}).reset_index()
lookups_df.columns=['id_name','id', 'id_index']

In [14]:
lookups_df.to_pickle('lookup_report_2016_09_01-2016_12_01.pkl')

Produce histogram of impressions by each id type
sort by impressions
plot ctr and cvr


In [17]:
a1=df_mg.groupby('Brand')[['Impressions','Clicks','Conversions', 'Cross-device conv.', 'Total conv. value']].sum()

In [18]:
a1['CTR']=a1['Clicks'].div(a1['Impressions'])
a1['CVR']=a1['Conversions'].div(a1['Clicks'])


In [21]:
a1.sort_values('Impressions',ascending=False, inplace=True)

In [22]:
a1.head()

,Impressions,Clicks,Conversions,Cross-device conv.,Total conv. value,CTR,CVR
Brand,,,,,,,
19,5653465.0,132052.0,674.0,6.0,186885.390566,0.023358,0.005104
21,2229866.0,36833.0,674.0,0.0,146943.010303,0.016518,0.018299
58,2198560.0,31683.0,452.0,0.0,134021.030163,0.014411,0.014266
63,1895228.0,19856.0,204.0,0.0,118038.810158,0.010477,0.010274
40,1724266.0,24891.0,301.0,0.0,93540.389893,0.014436,0.012093


In [23]:
lookups_df1=lookups_df.set_index(['id_name','id_index'])

In [39]:
x=lookups_df1.xs('Brand').id

In [40]:
x.head()

id_index
1085            --
359     0039 italy
479       120%lino
1021    4 barra 12
1053          4lou
Name: id, dtype: object

In [43]:
x.iloc[21]

'aeronautica'

In [37]:
lookups_df1.loc['Brand',19]

id    wellensteyn
Name: (Brand, 19), dtype: object

In [26]:
x.head()

,id
id_index,
1085,--
359,0039 italy
479,120%lino
1021,4 barra 12
1053,4lou


In [54]:
a1['Brand_name']=a1.reset_index()['Brand'].map(lookups_df1.xs('Brand').id)

In [59]:
a2=a1.reset_index()

In [58]:
a2.head()

0    19
1    21
2    58
3    63
4    40
Name: Brand, dtype: int64

In [60]:
a2['Brand_name']=a2['Brand'].map(lookups_df1.xs('Brand').id)

In [61]:
a2.head()

,Brand,Impressions,Clicks,Conversions,Cross-device conv.,Total conv. value,CTR,CVR,Brand_name
0,19,5653465.0,132052.0,674.0,6.0,186885.390566,0.023358,0.005104,wellensteyn
1,21,2229866.0,36833.0,674.0,0.0,146943.010303,0.016518,0.018299,marc o'polo
2,58,2198560.0,31683.0,452.0,0.0,134021.030163,0.014411,0.014266,hugo
3,63,1895228.0,19856.0,204.0,0.0,118038.810158,0.010477,0.010274,moncler
4,40,1724266.0,24891.0,301.0,0.0,93540.389893,0.014436,0.012093,boss


In [63]:
a2.describe()

/home/sean/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Brand,Impressions,Clicks,Conversions,Cross-device conv.,Total conv. value,CTR,CVR
count,1144.00000,1.144000e+03,1144.000000,1144.000000,1144.000000,1144.000000,1138.000000,1032.000000
mean,571.50000,5.794697e+04,887.131119,14.180070,0.076923,2984.878167,0.016644,inf
std,330.38866,2.481357e+05,4698.710545,48.244519,0.865005,11579.911007,0.015730,NaN
min,0.00000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,285.75000,4.690000e+02,6.000000,0.000000,0.000000,0.000000,NaN,NaN
50%,571.50000,2.987000e+03,45.500000,1.000000,0.000000,77.400002,NaN,NaN
75%,857.25000,2.398975e+04,335.500000,7.000000,0.000000,1514.622499,NaN,NaN
max,1143.00000,5.653465e+06,132052.000000,674.000000,17.000000,186885.390566,0.263158,inf


1144 brands, max 5 million impressions

In [73]:
a2[:50].plot(x='Brand_name',y=['Impressions','CTR','CVR'],subplots=True,kind='bar')

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f8a4fbcdcd0>,
       <matplotlib.axes._subplots.AxesSubplot object at 0x7f8a4f34f5d0>], dtype=object)

In [65]:
import matplotlib.pyplot as plt

In [66]:
f=plt.figure()

In [70]:
z=plt.subplot('311')

In [62]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [46]:
ll=lookups_df1.xs('Brand').id

In [44]:
a2=a1.head(20).copy()

In [51]:
a3=a2.reset_index()

In [52]:
a3['Brand_name']=a3['Brand'].map(ll)

In [53]:
a3

,Brand,Impressions,Clicks,Conversions,Cross-device conv.,Total conv. value,CTR,CVR,Brand_name
0,19,5653465.0,132052.0,674.0,6.0,186885.390566,0.023358,0.005104,wellensteyn
1,21,2229866.0,36833.0,674.0,0.0,146943.010303,0.016518,0.018299,marc o'polo
2,58,2198560.0,31683.0,452.0,0.0,134021.030163,0.014411,0.014266,hugo
3,63,1895228.0,19856.0,204.0,0.0,118038.810158,0.010477,0.010274,moncler
4,40,1724266.0,24891.0,301.0,0.0,93540.389893,0.014436,0.012093,boss
5,8,1666476.0,14575.0,296.0,0.0,64470.710548,0.008746,0.020309,paul green
6,38,1630998.0,19760.0,134.0,8.0,60419.470354,0.012115,0.006781,burberry
7,57,1313391.0,18037.0,106.0,0.0,61218.799752,0.013733,0.005877,woolrich
8,2,1232592.0,19081.0,338.0,0.0,80710.010141,0.015480,0.017714,boss orange
9,13,1200424.0,14882.0,169.0,0.0,21111.800037,0.012397,0.011356,camp david


In [29]:
a1.head()

,Impressions,Clicks,Conversions,Cross-device conv.,Total conv. value,CTR,CVR,Brand_name
Brand,,,,,,,,
19,5653465.0,132052.0,674.0,6.0,186885.390566,0.023358,0.005104,cmp
21,2229866.0,36833.0,674.0,0.0,146943.010303,0.016518,0.018299,joop!
58,2198560.0,31683.0,452.0,0.0,134021.030163,0.014411,0.014266,monari
63,1895228.0,19856.0,204.0,0.0,118038.810158,0.010477,0.010274,dior
40,1724266.0,24891.0,301.0,0.0,93540.389893,0.014436,0.012093,cambio


In [63]:
lookups['Category (1st level)']

{' --': 9,
 'Baby & Kleinkind': 7,
 'Bekleidung & Accessoires': 0,
 'B\xc3\xbcrobedarf': 5,
 'Elektronik': 6,
 'Gesundheit & Sch\xc3\xb6nheit': 1,
 'Heim & Garten': 3,
 'Kunst & Unterhaltung': 8,
 'Religion & Feierlichkeiten': 10,
 'Sportartikel': 2,
 'Taschen & Gep\xc3\xa4ck': 4}

In [73]:
a1.head()

,level_0,level_1,0
0,Ad group,0039 italy,380
1,Ad group,120%lino,515
2,Ad group,4 barra 12,1373
3,Ad group,4lou,1445
4,Ad group,7 for all mankind,461


In [45]:
dtype

{u'Ad group': u'np.uint16',
 u'Ad group ID': u'np.uint16',
 u'Brand': u'np.uint16',
 u'CTR': u'np.float32',
 u'Campaign': u'np.uint16',
 u'Campaign ID': u'np.uint16',
 u'Category (1st level)': u'np.uint16',
 u'Category (2nd level)': u'np.uint16',
 u'Category (3rd level)': u'np.uint16',
 u'Category (4th level)': u'np.uint16',
 u'Category (5th level)': u'np.uint16',
 u'Click share': u'str',
 u'Clicks': u'np.uint16',
 u'Conv. rate': u'np.float32',
 u'Conversions': u'np.uint16',
 u'Cost': u'np.float32',
 u'Country/Territory': u'np.uint16',
 u'Cross-device conv.': u'np.uint16',
 u'Custom label 0': u'np.uint16',
 u'Custom label 1': u'np.uint16',
 u'Custom label 2': u'np.uint16',
 u'Custom label 3': u'np.uint16',
 u'Custom label 4': u'np.uint16',
 u'Device': u'np.uint8',
 u'Impressions': u'np.uint16',
 u'Item Id': u'np.uint32',
 u'Product type (1st level)': u'np.uint16',
 u'Product type (2nd level)': u'np.uint16',
 u'Product type (3rd level)': u'np.uint16',
 u'Product type (4th level)': u'np.

In [44]:
converters

{u'Ad group': <function __main__.lookup>,
 u'Ad group ID': <function __main__.lookup>,
 u'Brand': <function __main__.lookup>,
 'CTR': <function __main__.<lambda>>,
 u'Campaign': <function __main__.lookup>,
 u'Campaign ID': <function __main__.lookup>,
 u'Category (1st level)': <function __main__.lookup>,
 u'Category (2nd level)': <function __main__.lookup>,
 u'Category (3rd level)': <function __main__.lookup>,
 u'Category (4th level)': <function __main__.lookup>,
 u'Category (5th level)': <function __main__.lookup>,
 'Conv. rate': <function __main__.<lambda>>,
 u'Country/Territory': <function __main__.lookup>,
 u'Custom label 0': <function __main__.lookup>,
 u'Custom label 1': <function __main__.lookup>,
 u'Custom label 2': <function __main__.lookup>,
 u'Custom label 3': <function __main__.lookup>,
 u'Custom label 4': <function __main__.lookup>,
 u'Device': <function __main__.lookup>,
 u'Item Id': <function __main__.lookup>,
 u'Product type (1st level)': <function __main__.lookup>,
 u'P

In [5]:
df1=pd.read_csv('report_2016_09_01-2016_12_01.csv', nrows=2000000)

/home/sean/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:

lookups['a']={}
a=do_lookup('a', np.uint16)

In [28]:
a('hello')
a('hi')

1

In [29]:
lookups

{'a': {'hello': 0, 'hi': 1}}

In [6]:
df1['Item Id'].max()

'9142330195969'

In [20]:
def cast_categ(df_inp, cat_types):
    for c in cat_types:
        df_inp[c]=df_inp[c].astype('category')

In [48]:
cast_categ(df1,cat_types)

In [30]:
df1.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Ad group ID,2e+06,NaN,NaN,NaN,3.33161e+10,1.96886e+09,1.76012e+10,3.35848e+10,3.35849e+10,3.35849e+10,4.38938e+10
Ad group,2000000,1539,marc o'polo,82865,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brand,2000000,1114,marc o'polo,84825,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Campaign ID,2e+06,NaN,NaN,NaN,6.33312e+08,6.86819e+07,2.01385e+08,6.44233e+08,6.44233e+08,6.44233e+08,7.04892e+08
Campaign,2000000,6,camato Designer,1256943,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (1st level),2000000,11,Bekleidung & Accessoires,1702086,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (2nd level),2000000,34,Bekleidung,1351333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (3rd level),2000000,69,Überbekleidung,439678,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (4th level),2000000,93,--,1137782,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category (5th level),2000000,55,--,1847175,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
_.to_clipboard()

NoClipboardProgramError: Pyperclip requires the gtk, PyQt4, or PySide module installed, or either the xclip or xsel command.

In [51]:
cnt=df1.groupby([ 'Ad group', 'Ad group ID']).size()

In [52]:
cnt.shape

(2576,)

In [45]:
cnt.reset_index().describe()

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

In [53]:
cnt1=cnt.groupby(level='Ad group').size()

In [54]:
cnt1[cnt1>1]

Ad group
0039 italy                            2
120%lino                              2
4lou                                  2
7 for all mankind                     2
81hours                               2
a beautiful story                     2
a-k-r-i-s-                            2
abro                                  2
acca kappa                            2
acqua di parma                        2
adenauer & co                         2
adidas                                2
adidas by raf simons                  2
adidas by stella mccartney            2
adidas originals                      2
adidas stellasport                    2
aeronautica militare                  2
ag jeans                              2
agl attilio giusti leombruni          2
agl-attilio giusti leombruni          2
aglini                                2
agnona                                2
aigner                                2
airfield                              2
airforce                       

In [38]:
cnt.head()

Ad group         Ad group ID
Marken_Barts     17602163553      125
Marken_Boss      17601240513    12412
Marken_Cambio    17602161273      424
Marken_Desigual  17922688713      174
Marken_Doell     17602162953       57
dtype: int64

In [36]:
cnt.reset_index()['Ad group'].isnull().sum()

0

In [32]:
df_test[['Ad group ID', 'Ad group', 'Brand']].sample(50)

,Ad group ID,Ad group,Brand
99468,17601222753,Marken_Marc-Cain,marccain
9072,17601240513,Marken_Boss,boss
71515,17601229593,Marken_Marc-O'Polo,marc o'polo
24755,17601222753,Marken_Marc-Cain,marccain
75791,17601222753,Marken_Marc-Cain,marccain
32600,17602244913,Marken_Rest,strenesse
22092,17601240513,Marken_Boss,boss
24520,17602244913,Marken_Rest,raffaello rossi
29028,17601230193,Marken_Drykorn,drykorn
94651,17601229593,Marken_Marc-O'Polo,marc o'polo pure


In [10]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
Clicks,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Conversions,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Total conv. value,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Cost,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Country/Territory,100000.0,2276.00000,0.000000,2276.0,2276.0,2276.0,2276.0,2276.0
Cross-device conv.,100000.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Impressions,100000.0,0.00003,0.005477,0.0,0.0,0.0,0.0,1.0


In [7]:
dt

,column_name,data_type
0,Ad group ID,category
1,Ad group,category
2,Brand,category
3,Campaign ID,category
4,Campaign,category
5,Category (1st level),category
6,Category (2nd level),category
7,Category (3rd level),category
8,Category (4th level),category
9,Category (5th level),category
